# API's

## Application Programming Interface

What is it?
- An intermediary to communicate with a server

Why do we use them?
- To request data
- To trigger jobs

```mermaid
graph RL
  subgraph User
    A((User))
  end

  subgraph API
    B{{API}}
  end

  subgraph Server
    C((Server))
  end

  A -->|Sends Request| B
  B -->|Relays Request| C
  C -->|Returns Result or Status| B
  B -->|Sends Response| A

```

### Common request types:
- GET (used to retreive data from a server)
- POST (sends data to an API, either creating or updating a resource)

### Common responses:
- 2XX 😎
- 4XX 🤬
- 5XX 🙄

# API Examples

1) BigQuery
2) Composer
3) Other

## BigQuery API

Prerequisites:

Set Default Credentials: 
```gcloud auth application-default login```

This will save your credentials and allow you to access BigQuery (presuming you have the correct permissions)

In [1]:
from google.cloud import bigquery

### Reading from BQ

In [2]:
client = bigquery.Client()

In [3]:
sql = """
    SELECT name, SUM(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY name
    ORDER BY count DESC
    LIMIT 10
"""

In [4]:
df = client.query(sql).to_dataframe()

In [5]:
df

,name,count
0,James,5054074
1,John,4910976
2,Robert,4763102
3,Michael,4396482
4,William,3939912
5,Mary,3757625
6,David,3631489
7,Joseph,2554530
8,Richard,2550606
9,Charles,2299419


```mermaid
graph RL
  subgraph User
    A((User))
  end

  subgraph BigQueryAPI
    B{{BigQuery API}}
  end

  subgraph Server
    C{{BigQuery}}
  end

  A -->|Sends Query Request| B
  B -->|Tells BQ to process job| C
  C -->|Sends Result| B
  B -->|Returns Dataframe| A
```

### Pandas GBQ

An alternative lightweight way to read and write data to BigQuery

In [10]:
import pandas as pd
import pandas_gbq

In [11]:
SQL = """
    SELECT name, SUM(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY name
    ORDER BY count DESC
    LIMIT 10
"""

In [12]:
df = pd.read_gbq(SQL, project_id='jlr-dl-cat')

## Triggering Airflow DAGs

Airflow DAGs are commonly triggered using cloud functions that communicate with composer via an API 

In [6]:
import google.auth
from google.auth.transport.requests import AuthorizedSession

To get airflow uri:
```
gcloud composer environments describe jlr-dl-cat-schumann --location europe-west2
```

In [9]:
# Getting credentials
AUTH_SCOPE = "https://www.googleapis.com/auth/cloud-platform"
CREDENTIALS, _ = google.auth.default(scopes=[AUTH_SCOPE])

# Defining info about our composer environment and DAG
airflow_uri = 'https://878d6511cdc24c9b95a34f8450e94025-dot-europe-west2.composer.googleusercontent.com'
dag_id = 'airflow_monitoring'
data = {'conf': {}}
endpoint = f"api/v1/dags/{dag_id}/dagRuns"
request_url = f"{airflow_uri}/{endpoint}"

In [8]:
# Making API call
authed_session = AuthorizedSession(CREDENTIALS)
authed_session.request(url=request_url, method='POST', json=data)

<Response [404]>

```mermaid
graph RL
  subgraph User
    A((User))
  end

  subgraph ComposerAPI
    B{{Composer API}}
  end

  subgraph Airflow Server
    C{{Airflow DAG}}
  end

  A -->|Sends POST Request| B
  B -->|Triggers DAG Execution| C
  B -->|Returns Status e.g. 200| A
```

## Other

Google API library: https://console.cloud.google.com/apis/library?hl=en-GB

For PD: https://rapidapi.com/hub